# Before everything
the task 1 is to find the top 10 teams, which i think is a prediction problem, so first i will just do a small test without cross validation, so i will just use cbb15-17 as training set and cbb18 as testing set, as i noticed that cbb19 has 2 more columns which could be 2 more teams.

# Review of normal procedure
1. understand all features and categorize them into numerical values and categoritical values
2. feature engineering: analyse features by visualizing them and drop the most unrelevant features and generate new representative features based on old features
3. feature engineering: 7C methods to pre-perpare datasets(cleanse, convert, clear...)
4. create basemodel to make first-level prediction
5. ensemble and stack: use the output of first-level prediction to make second-level prediction
6. score the model 

In [ ]:
#import libraries 
import numpy as np
import pandas as pd 
import seaborn as sns

#ignore warning
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold
#use xgb for the second classifier
import xgboost as xgb

In [ ]:
#read dataset
cbb = pd.read_csv('../input/college-basketball-dataset/cbb.csv')
cbb.head(10)

In [ ]:
#get all rows without year == 2019
cbb_ = cbb[~(cbb.YEAR == 2019)]
cbb_.head(10)



# Understand all features
just copied it from the description


TEAM: Name of all schools participated into the contest 学校的名字
**looks like irrelevant from the first sight but
1. maybe the school names also indicate the location and has some correlation with CONF
**

CONF: The Athletic Conference in which the school participates in (A10 = Atlantic 10, ACC = Atlantic Coast Conference, AE = America East, Amer = American, ASun = ASUN, B10 = Big Ten, B12 = Big 12, BE = Big East, BSky = Big Sky, BSth = Big South, BW = Big West, CAA = Colonial Athletic Association, CUSA = Conference USA, Horz = Horizon League, Ivy = Ivy League, MAAC = Metro Atlantic Athletic Conference, MAC = Mid-American Conference, MEAC = Mid-Eastern Athletic Conference, MVC = Missouri Valley Conference, MWC = Mountain West, NEC = Northeast Conference, OVC = Ohio Valley Conference, P12 = Pac-12, Pat = Patriot League, SB = Sun Belt, SC = Southern Conference, SEC = South Eastern Conference, Slnd = Southland Conference, Sum = Summit League, SWAC = Southwestern Athletic Conference, WAC = Western Athletic Conference, WCC = West Coast Conference) 
**looks like this CONF is geographically related and symbolize which sector does which school plays in**

G: Number of games played
**the more games played, the more skilled**

W: Number of games won
**the more games won, the more energetic players are and higher the spirit**

ADJOE: Adjusted Offensive Efficiency (An estimate of the offensive efficiency (points scored per 100 possessions) a team would have against the average Division I defense)
**seems to me its 进攻效率，每100次控球进攻得分，对于平均defense 来说
so the more points scored, the more likely the team to be categorized as a offensive team, which means they play more aggresive
**

ADJDE: Adjusted Defensive Efficiency (An estimate of the defensive efficiency (points allowed per 100 possessions) a team would have against the average Division I offense)
**seems to me its 防守效率，每100次(对方)控球我们被得的分，对于平均offense 来说
(here im not sure if points allowed means the points scored by offense or the points we managed to prevent, let me assue its the first situation) so the higher ADJDE, the worse we play defense
so the more points scored, the more likely the team to be categorized as a defensive team, which means they play more defensive
**

BARTHAG: Power Rating (Chance of beating an average Division I team)
**general score of power**

EFG_O: Effective Field Goal Percentage Shot
**有效场均命中率**

EFG_D: Effective Field Goal Percentage Allowed
**对方有效场均命中率**

TOR: Turnover Percentage Allowed (Turnover Rate)
**失误率**

TORD: Turnover Percentage Committed (Steal Rate)
**抢断率**

ORB: Offensive Rebound Percentage
**场均进攻篮板**

DRB: Defensive Rebound Percentage
**场均防守篮板**

FTR : Free Throw Rate (How often the given team shoots Free Throws)
**罚球率**

FTRD: Free Throw Rate Allowed
**对方罚球率**

2P_O: Two-Point Shooting Percentage
**2分命中率**

2P_D: Two-Point Shooting Percentage Allowed
**对方2分出手率**

3P_O: Three-Point Shooting Percentage
**3分命中率**

3P_D: Three-Point Shooting Percentage Allowed
**对方3分出手率**

ADJ_T: Adjusted Tempo (An estimate of the tempo (possessions per 40 minutes) a team would have against the team that wants to play at an average Division I tempo)
**控球率**

WAB: Wins Above Bubble (The bubble refers to the cut off between making the NCAA March Madness Tournament and not making it)
**胜利基于现场燃爆！**

POSTSEASON: Round where the given team was eliminated or where their season ended (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)
**总决赛第几强**

SEED: Seed in the NCAA March Madness Tournament
**种子选手**

YEAR: Season
**赛季**

After overviewing the meaning of each feature, lets take a look into details of the parameters of each feature

In [ ]:
for i,el in enumerate(cbb_):
    print(el +'   is')
    print(pd.unique(cbb_[el]))
    print('-'*30)

Visualization work maybe for the version 0.2 here i will just cleanse, mapping and generate new features

decision:
1. mapping all string elements with integer
2. fill nan with differnt number 
3. delete 'G' and 'W' generate new feature winrate


p.s. from the visualization
1. here i fill the nan with big numbers, later should be changed to mean of other values
2. mapping conf may be first sorted by the win rate
3. calculate win rate= win times/game times

In [ ]:

# generate winrate
cbb_['WINRATE'] = cbb_['W']/cbb_['G']
# generate winrate above bubble
cbb_['WINRATE_BUBBLE'] = cbb_['WAB']/cbb_['G']
#drop W ,G ,team name, Winsabovebubble
cbb_.drop(columns = ['W','G','TEAM','WAB'],inplace = True)
cbb_.columns.values

In [ ]:
#mapping POSTSEASON
pos_mapping = {'Champions':0,'2ND':1,'F4':2,'E8':3,'S16':4,'R32':5,'R64':6,'R68':7}
cbb_['POSTSEASON'] = cbb_['POSTSEASON'].map(pos_mapping)
cbb_['POSTSEASON'] = cbb_['POSTSEASON'].fillna(8)

#mapping SEED, punish team without seed
cbb_['SEED'] = cbb_['SEED'].fillna(30)

#mapping CONF
conf_mapping = {'WCC':0,'ACC':1 ,'B10':2 ,'SEC':3, 'B12' :4,'Amer':5, 'BE' :6,'MAC':7, 'SC':8, 'MWC':9, 'A10':10, 'P12':11,
 'OVC': 12,'WAC':13, 'BSth':14, 'BW':15, 'AE': 16,'CAA':17, 'Ivy':18, 'Horz':19,'SB':20, 'CUSA':21, 'Pat':22, 'BSky':23,
 'MVC': 24,'Slnd':25, 'Sum' :26,'MAAC': 27,'SWAC':28, 'NEC':29 ,'MEAC':30, 'ASun':31}
cbb_['CONF'] = cbb_['CONF'].map(conf_mapping)
cbb_['CONF'] = cbb_['CONF'].fillna(100)

#no need to map school name cuz too big and corupt the correlation just drop it

#after preparation finally to split data
train_set = cbb_[~(cbb_.YEAR == 2018)]
test_set = cbb_[(cbb_.YEAR == 2018)]

#feature selection in later version
train_set.head(10)

In [ ]:
"""
#just for test visualizing
test_cbb = cbb

#plot a multiplot 

df = pd.DataFrame({'g':test_cbb['G'],'P':test_cbb['POSTSEASON']})
sns.jointplot(x='g',y='P',data = df)
"""

In [ ]:
#see heatmap the correlation between features
colormap = plt.cm.RdBu
plt.figure(figsize=(24,22))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train_set.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
#just copied
# Some useful parameters which will come in handy later on
ntrain = train_set.shape[0]#number of train sets
ntest = test_set.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits = NFOLDS, shuffle = False, random_state=None)


# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [ ]:
#create Out of Folder function
def get_oof(clf, x_train, y_train, x_test):
    #训练集的样本数
    oof_train = np.zeros((ntrain,))
    #测试集的样本数
    oof_test = np.zeros((ntest,))
    #(5，测试集的样本数)
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    #对于大训练集分成5分，细分成为4个小训练集和1个小测试集
    for i,(train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]#训练集x
        y_tr = y_train[train_index]#训练集y
        x_te = x_train[test_index]#测试集x

        clf.train(x_tr, y_tr)#训练 小训练集

        oof_train[test_index] = clf.predict(x_te)#对小测试集 predict
        oof_test_skf[i, :] = clf.predict(x_test)#对大测试集 predict

    oof_test[:] = oof_test_skf.mean(axis=0)#kfold 之后对于大测试集的 predict 求mean
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)#only one column and all values are in one column

In [ ]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [ ]:
# Create 5 objects that represent our 4 models
#rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

# training

In [ ]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train_set['POSTSEASON'].ravel()#展开成1d
train_set = train_set.drop(['POSTSEASON'], axis=1)
x_train = train_set.values # Creates an array of the train data

y_test = test_set['POSTSEASON'].ravel()
test_set = test_set.drop(['POSTSEASON'], axis=1)
x_test = test_set.values # Creats an array of the test data

# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
#rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

In [ ]:
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)
#svc_feature = svc.feature_importances(x_train,y_train)

In [ ]:
x_train = np.concatenate(( et_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

In [ ]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)
print(predictions)

In [ ]:
from sklearn import metrics
print(y_test)
score = metrics.accuracy_score(y_test, predictions)
print(f"Test score: {score}")

In [ ]:
#create output csv
"""
# Generate Submission File 
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })
StackingSubmission.to_csv("StackingSubmission.csv", index=False)
"""
